In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import datetime
import numpy as np

Some station reserach stuff
https://www1.nyc.gov/assets/planning/download/pdf/plans-studies/sustainable-communities/bmn/bronx_metro_report/fordham.pdf

Some random references:

http://www.subchat.com/read.asp?Id=892212

https://web.archive.org/web/20100613000942/http://www.railroad.net/forums/viewtopic.php?f=67&t=68068

http://www.railroad.net/forums/search.php?st=0&sk=t&sd=d&sr=posts&keywords=ridership+station&fid%5B%5D=67&start=15

1980 NJT Map
http://www.subchat.com/read.asp?Id=693001  http://www.subchat.com/read.asp?Id=693400

List of Long Island rail Road Stations 

https://en.wikipedia.org/wiki/List_of_Long_Island_Rail_Road_stations

Some awesome cartography stuff

http://www.vanshnookenraggen.com/_index/2017/05/a-complete-and-geographically-accurate-nyc-subway-track-map/

In [2]:
def bs_parse_table(url,table_name):
    soup = BeautifulSoup(open(url), "html5lib")
    soup_table = soup.find('table', id=table_name)
    return soup_table

In [3]:
def extract_header_from_soup(soup_table):
    header_array = []
    for header_values in soup_table.find_all('th'):
        header_array.append(header_values.get_text().strip())
    switch = header_array.pop()
    header_array.append('pct_change')
    header_array.append(switch)
    header_array.append('borough')
    return header_array

In [4]:
def listify_table(soup_table):
    table_list = []

    for row in soup_table.find_all('tr'):
        current_row_list = []
        for cell in row.find_all('td'):
            if len(cell) > 0 and 'alt' in str(cell.contents[0]):
                station_info = []
                station_info.append(cell.get_text())
                other_name_tags = cell.find_all("img")
                if len(other_name_tags)>0:
                    for alt_text_in in other_name_tags:
                        station_info.append(alt_text_in.get('alt'))
                current_row_list.append(station_info)
            else:
                current_row_list.append(cell.get_text())
        table_list.append(current_row_list)
        
    new_table = []
    for row in table_list:
        new_row = row.copy()
        if len(row)==1:
            current_borough = row[0].lower().replace(' ','_')
        if len(row)>0 and row[0]=='Brooklyn':
            current_borough = 'city_wide_stuff'
        if len(row)==10 and len(row[4])>0:
            new_row.append(current_borough)
            new_table.append(new_row)        
    return new_table

In [5]:
def station_information_parser(new_table):
    stations = []
    line_details = []
    for row in new_table:
        current_subway_lines = []
        if isinstance(row[0], str):
            current_subway_lines=[]
            stations.append('')
            line_details.append(current_subway_lines)
        else:
            for i in range (0,len(row[0])):
                if i==0:
                    cleaned_name = re.sub( '\s+', ' ', row[0][i]).strip()
                    stations.append(cleaned_name)
                else:
                    line_number = row[0][i].replace('subway','').replace('icon','').replace('train','').strip()
                    current_subway_lines.append(line_number)
            line_details.append(current_subway_lines)
    return stations, line_details

In [9]:
def generate_parsed_df(url,table_name):
    bs_table_object = bs_parse_table(url,table_name)
    header_array = extract_header_from_soup(bs_table_object)
    table_array = listify_table(bs_table_object)
    ridership_output = pd.DataFrame(table_array,columns = header_array)
    station_line = station_information_parser(table_array)
    ridership_output.loc[:,('station_name')] = station_line[0]
    ridership_output.loc[:,('lines_stopping_here')] = station_line[1]
    return ridership_output

In [10]:
path_2011_2016_annual = "../raw/mta_facts_and_figures_annual_station_ridership_02_22_2018.htm"
df_2011_2016_annual = generate_parsed_df(path_2011_2016_annual,"subway")

In [11]:
path_2007_2012_annual = "../raw/mta_facts_and_figures_annual_station_ridership_07_07_2013.htm"
df_2007_2012_annual = generate_parsed_df(path_2007_2012_annual,"subway")

In [12]:
#http://web.mta.info/nyct/facts/ridership/ridership_sub.htm
#path_2011_2016_average_weekday = "../raw/mta_facts_and_figures_average_weekday_station_ridership_02_23_2018.htm"
#df_2011_2016_average_weekday = generate_parsed_df(path_2011_2016_average_weekday,"subway")

In [13]:
#http://web.mta.info/nyct/facts/ridership/ridership_sub.htm
#path_2007_2012_average_weekday = "../raw/mta_facts_and_figures_average_weekday_station_ridership_07_07_2013.htm"
#df_2007_2012_average_weekday = generate_parsed_df(path_2007_2012_average_weekday,"subway")

In [14]:
#df_2011_2016_average_weekday.to_csv('../cleaned/2011_2016_average_weekday.csv',index=False)
#df_2007_2012_average_weekday.to_csv('../cleaned/2007_2012_average_weekday.csv',index=False)

In [15]:
df_2007_2012_annual.columns

Index(['Station (alphabetical by borough)', '2007', '2008', '2009', '2010',
       '2011', '2012', '2011-12 Change', 'pct_change', '2012 Rank', 'borough',
       'station_name', 'lines_stopping_here'],
      dtype='object')

In [16]:
df_2011_2016_annual.columns

Index(['Station (alphabetical by borough)', '2011', '2012', '2013', '2014',
       '2015', '2016', '2015-2016 Change', 'pct_change', '2016 Rank',
       'borough', 'station_name', 'lines_stopping_here'],
      dtype='object')

In [17]:
df_2011_2016_annual[:-6]['2016'].str.replace(',','').astype(int).sum()

1755724826

In [18]:
df_2011_2016_annual.drop(['2015-2016 Change','pct_change','2016 Rank'],axis=1).head()

,Station (alphabetical by borough),2011,2012,2013,2014,2015,2016,borough,station_name,lines_stopping_here
0,"[149 St-Grand Concourse , 2 subway, 4 sub...","4,169,699","4,104,847","4,427,399","4,536,888","4,424,754","4,381,900",the_bronx,149 St-Grand Concourse,"[2, 4, 5]"
1,"[161 St-Yankee Stadium , B subway, D subw...","8,605,893","8,831,015","8,766,012","8,961,029","8,922,188","8,784,407",the_bronx,161 St-Yankee Stadium,"[B, D, 4]"
2,"[167 St , 4 subway]","2,978,748","3,039,634","3,081,534","3,067,345","3,180,274","3,179,087",the_bronx,167 St,[4]
3,"[167 St , B subway, D subway]","2,952,368","2,972,535","3,091,289","3,245,977","3,295,032","3,365,748",the_bronx,167 St,"[B, D]"
4,"[170 St , 4 subway]","2,954,573","2,980,786","2,961,575","2,941,958","3,045,205","3,038,777",the_bronx,170 St,[4]


In [19]:
hack_for_unreported = pd.DataFrame(['138 St-Grand Concourse','917,911','939,284','957,984','1,033,559','1,056,380',\
 '1,070,024','the_bronx','138 St-Grand Concourse',['4','5']]).transpose()

In [20]:
hack_for_unreported.columns=['Station (alphabetical by borough)',\
                             '2011','2012','2013','2014','2015','2016','borough','station_name','lines_stopping_here']

In [21]:
hack_for_unreported

,Station (alphabetical by borough),2011,2012,2013,2014,2015,2016,borough,station_name,lines_stopping_here
0,138 St-Grand Concourse,"917,911","939,284","957,984","1,033,559","1,056,380","1,070,024",the_bronx,138 St-Grand Concourse,"[4, 5]"


In [22]:
df_2011_2016_annual_enriched=pd.concat([hack_for_unreported,\
        df_2011_2016_annual.drop(['2015-2016 Change','pct_change','2016 Rank'],axis=1)])
        

In [23]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    display(x)
    pd.reset_option('display.max_rows')

In [24]:
df_2007_2012_annual.query('station_name=="49 St"')

,Station (alphabetical by borough),2007,2008,2009,2010,2011,2012,2011-12 Change,pct_change,2012 Rank,borough,station_name,lines_stopping_here
276,"[49 St , N train icon, Q train icon, R train...","7,976,466","8,106,715","7,817,461","8,139,011","8,121,552","8,186,576","+65,024",+0.8%,41,manhattan,49 St,"[N, Q, R]"


In [25]:
df_2011_2016_annual_enriched.query('station_name=="49 St"')

,Station (alphabetical by borough),2011,2012,2013,2014,2015,2016,borough,station_name,lines_stopping_here
276,"[49 St , N subway, W subway, R subway]","8,121,552","8,186,576","8,155,010","8,544,694","8,029,988","8,754,627",manhattan,49 St,"[N, W, R]"


In [26]:
all_years_annual = pd.merge(df_2007_2012_annual\
                    .drop(['2011-12 Change','pct_change','2012 Rank',\
                           'Station (alphabetical by borough)','station_name',\
                           'borough','lines_stopping_here'],axis=1),\
         df_2011_2016_annual_enriched,\
         left_on=['2011','2012'],right_on=['2011','2012'],how="outer")


In [27]:
df_2011_2016_annual_enriched.query('station_name=="Cortlandt St"')

,Station (alphabetical by borough),2011,2012,2013,2014,2015,2016,borough,station_name,lines_stopping_here
315,"[Cortlandt St , 1 subway]",0,0,0,0,0,0,manhattan,Cortlandt St,[1]
316,"[Cortlandt St , R subway, W subway]","1,640,638","2,062,289","1,601,732","1,500,040","2,713,532","4,270,036",manhattan,Cortlandt St,"[R, W]"


In [28]:
all_years_annual = all_years_annual.rename(columns={'Station (alphabetical by borough)':'station_full_name'})

In [29]:
all_borough_fix = all_years_annual[:-7].copy()
all_borough_fix.loc[:,('borough')]=\
    all_borough_fix.loc[:,('borough')].str.replace('city_wide_stuff','brooklyn')

In [30]:
all_years_annual_fix = pd.concat([all_borough_fix,all_years_annual[-7:].copy()])

In [31]:
all_years_annual_fix.borough.unique()

array(['the_bronx', 'brooklyn', 'manhattan', 'queens', 'city_wide_stuff'],
      dtype=object)

In [32]:
def borough_remap(in_string):
    map_dict = {'manhattan':'M','the_bronx':'Bx','brooklyn':'B','queens':'Q','city_wide_stuff':'city_wide_stuff'}
    return map_dict[in_string]

In [33]:
all_years_annual_fix.loc[:,('borough')]=\
    all_years_annual_fix.loc[:,('borough')].apply(borough_remap)

In [34]:
def line_string_maker(input_list):
    i=0
    output_string = ''
    for item in input_list:
        if i==0:
            output_string=output_string+str(item)
            i=i+1
        else:
            output_string=output_string+'_'+str(item)
    return output_string

In [35]:
all_years_annual_fix.loc[:,('station_key')]= all_years_annual_fix.loc[:,('station_name')]\
    .str.replace(' /','_').str.replace(' ','_').str.replace('-','_')\
    +'|'+all_years_annual_fix.loc[:,('lines_stopping_here')].apply(line_string_maker)

In [36]:
all_years_annual_fix.loc[:,('line_designators_str')]=all_years_annual_fix.loc[:,('lines_stopping_here')]\
                                                          .apply(line_string_maker)

In [38]:
i=0
for column in range(2007,2017):
    columns_to_select = ['station_full_name','borough','station_name','lines_stopping_here',\
                         'station_key','line_designators_str']
    columns_to_select.append(str(column))
    current = all_years_annual_fix[columns_to_select].copy()
    current=current.rename(columns={str(column):'annual_ridership'})
    current.loc[:,('year')]=column
    if i==0:
        all_years_annual_fix_stacked = current.copy()
        i=i+1
    else:
        all_years_annual_fix_stacked=pd.concat([all_years_annual_fix_stacked,current.copy()])

In [39]:
all_years_annual_fix_stacked.loc[:,('annual_ridership')] = \
    all_years_annual_fix_stacked.loc[:,('annual_ridership')].fillna('0')\
    .str.replace(u'\xa0', '0')\
    .str.replace(',','').astype(int)

In [40]:
def year_parser_end(input_string):
    return pd.to_datetime(datetime.date(int(input_string),12,31))

In [41]:
all_years_annual_fix_stacked.loc[:,('date_standardized')] = \
    all_years_annual_fix_stacked.loc[:,('year')].apply(year_parser_end)

In [42]:
#all_years_annual_fix_stacked.to_csv(\
#    '../cleaned/stacked_ridership_2007_2016_short.csv',\
#    index=False)

In [43]:
all_years_annual_fix_stacked.head()

,station_full_name,borough,station_name,lines_stopping_here,station_key,line_designators_str,annual_ridership,year,date_standardized
0,138 St-Grand Concourse,Bx,138 St-Grand Concourse,"[4, 5]",138_St_Grand_Concourse|4_5,4_5,800595,2007,2007-12-31
1,"[149 St-Grand Concourse , 2 subway, 4 sub...",Bx,149 St-Grand Concourse,"[2, 4, 5]",149_St_Grand_Concourse|2_4_5,2_4_5,3112547,2007,2007-12-31
2,"[161 St-Yankee Stadium , B subway, D subw...",Bx,161 St-Yankee Stadium,"[B, D, 4]",161_St_Yankee_Stadium|B_D_4,B_D_4,7836990,2007,2007-12-31
3,"[167 St , 4 subway]",Bx,167 St,[4],167_St|4,4,2715327,2007,2007-12-31
4,"[167 St , B subway, D subway]",Bx,167 St,"[B, D]",167_St|B_D,B_D,2834640,2007,2007-12-31


In [44]:
total_values = []
for row in all_years_annual_fix_stacked.values:
    current = row
    for line in row[-6]:
#        current = np.append(current.copy(),line)
        total_values.append(np.append(current.copy(),line))
    
update_names = all_years_annual_fix_stacked.columns.tolist()
update_names.append('line')    

all_years_annual_fix_stacked_tall = pd.DataFrame(total_values,columns=update_names)

all_years_annual_fix_stacked_tall_w = all_years_annual_fix_stacked_tall.query('line=="W"').copy()
all_years_annual_fix_stacked_tall_non_w = all_years_annual_fix_stacked_tall.query('line!="W"').copy()
all_years_annual_fix_stacked_tall_w_n = \
        all_years_annual_fix_stacked_tall_w[all_years_annual_fix_stacked_tall_w["station_name"]\
            .isin(['23 St','28 St','8 St-New York University','Prince St'])].copy()
all_years_annual_fix_stacked_tall_w_q = \
        all_years_annual_fix_stacked_tall_w[~all_years_annual_fix_stacked_tall_w["station_name"]\
            .isin(['23 St','28 St','8 St-New York University','City Hall','Cortlandt St','Prince St','Rector St',\
                   'South Ferry /Whitehall St'])].copy()    

all_years_annual_fix_stacked_tall_w_q['line'] = all_years_annual_fix_stacked_tall_w_q['line'].str.replace('W','Q')
all_years_annual_fix_stacked_tall_w_n['line'] = all_years_annual_fix_stacked_tall_w_n['line'].str.replace('W','N')

all_years_annual_fix_stacked_tall = pd.concat([all_years_annual_fix_stacked_tall_non_w\
                                            ,all_years_annual_fix_stacked_tall_w_q
                                             , all_years_annual_fix_stacked_tall_w_n])

all_years_annual_fix_stacked_tall = all_years_annual_fix_stacked_tall.drop('station_full_name',axis=1)
import ast
def string_list_to_list(x):
    return ast.literal_eval(x)

all_years_annual_fix_stacked_tall.loc[:,('lines_stopping_here')]= \
    all_years_annual_fix_stacked_tall.loc[:,('lines_stopping_here')].astype(str)

all_years_annual_fix_stacked_tall = all_years_annual_fix_stacked_tall.drop_duplicates()
    
all_years_annual_fix_stacked_tall = \
    all_years_annual_fix_stacked_tall.rename(columns={'lines_stopping_here':'line_designators'})

In [45]:
all_years_annual_fix_stacked_tall.loc[:,('station_key_split')]=\
    all_years_annual_fix_stacked_tall.loc[:,('line')]+ \
    '|'+\
    all_years_annual_fix_stacked_tall.loc[:,('station_key')]

In [46]:
all_years_annual_fix_stacked_tall.head()

,borough,station_name,line_designators,station_key,line_designators_str,annual_ridership,year,date_standardized,line,station_key_split
0,Bx,138 St-Grand Concourse,"['4', '5']",138_St_Grand_Concourse|4_5,4_5,800595,2007,2007-12-31,4,4|138_St_Grand_Concourse|4_5
1,Bx,138 St-Grand Concourse,"['4', '5']",138_St_Grand_Concourse|4_5,4_5,800595,2007,2007-12-31,5,5|138_St_Grand_Concourse|4_5
2,Bx,149 St-Grand Concourse,"['2', '4', '5']",149_St_Grand_Concourse|2_4_5,2_4_5,3112547,2007,2007-12-31,2,2|149_St_Grand_Concourse|2_4_5
3,Bx,149 St-Grand Concourse,"['2', '4', '5']",149_St_Grand_Concourse|2_4_5,2_4_5,3112547,2007,2007-12-31,4,4|149_St_Grand_Concourse|2_4_5
4,Bx,149 St-Grand Concourse,"['2', '4', '5']",149_St_Grand_Concourse|2_4_5,2_4_5,3112547,2007,2007-12-31,5,5|149_St_Grand_Concourse|2_4_5


In [47]:
all_years_annual_fix_stacked_tall.to_csv(\
    '../cleaned/stacked_annual_ridership_web_2007_2016.csv',\
    index=False)